In [169]:
print("This model is meant to be run exclusively on CPU, not GPU using cuda. The changes are not drastic though to switch between the two, they require some conditional logic and pushing the parameters to device, but nothing huge. Loss is only calculated once, not averaged, this is a necessary optimization though")
print("This model is decoder only, so lacks encoder part of Attention is all you need paper, and lacks cross-attention")
print("This model is just a document completer, not a question answerer: need to fine-tune beyond the pretraining here to accomplish this")

This model is meant to be run exclusively on CPU, not GPU using cuda. The changes are not drastic though to switch between the two, they require some conditional logic and pushing the parameters to device, but nothing huge. Loss is only calculated once, not averaged, this is a necessary optimization though
This model is decoder only, so lacks encoder part of Attention is all you need paper, and lacks cross-attention
This model is just a document completer, not a question answerer: need to fine-tune beyond the pretraining here to accomplish this


In [170]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [171]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [172]:
enc = {ch:i for i, ch in enumerate(chars)} #iterators via enumerate
dec = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [enc[c] for c in s] #serves as our encoder to convert characters in a sequence to list of numbers
decode = lambda l: ''.join([dec[i] for i in l]) #conv list of integers into a string (decoder)
print(encode("hii there"))
print(decode(encode("hii there")))
#conversion both ways complete

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [173]:
print("above is a character level tokenizer scheme, small vocab of only 65 letters in the shakespeare dataset: but can use tiktoken by openAI or sentencepiece by google to create a more robust encoder/decoder scheme")

above is a character level tokenizer scheme, small vocab of only 65 letters in the shakespeare dataset: but can use tiktoken by openAI or sentencepiece by google to create a more robust encoder/decoder scheme


In [174]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
#print(data[:1000])

torch.Size([1115394]) torch.int64


In [175]:
#create a couple global vars for rest of book
n_embd = 128
dropout = 0.2
#number of embedding dimensions

In [176]:
cutoff = int(0.9*len(data))
train = data[:cutoff]
val = data[cutoff:]

In [177]:
context_window = 64
train[:context_window+1] #to enable offsets for prediction/target

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50])

In [178]:
x = train[:context_window]
y = train[1:context_window+1]
for i in range(context_window):
    context = x[:i+1]
    target = y[i]
    #print(f"input {context} target {target}")
print("trains context length times with varying context lengths from 1 to context length for training efficiency, but inference will be less efficient off many softmax PD predictions going to 'waste' for inference use case")

trains context length times with varying context lengths from 1 to context length for training efficiency, but inference will be less efficient off many softmax PD predictions going to 'waste' for inference use case


In [179]:
#batching for parallelization
#torch.manual_seed(1337)
batchsize = 32 #num parallel sequences being processed
cwbatch = 64 #context window max len
def get_batch(splitname):
    #create batches of data
    data = train if splitname == "train" else val
    idx = torch.randint(len(data) - cwbatch, (batchsize,))
    x =  torch.stack([data[i:i+cwbatch] for i in idx])
    y = torch.stack([data[i+1:i+1+cwbatch] for i in idx])
    return x, y
xbatch, ybatch = get_batch('train')
#print("inputs:")
#print(xbatch.shape)
#print(xbatch)
#print("targets:")
#print(ybatch.shape)
#print(ybatch)

print("______")
for b in range(batchsize):
    for t in range(cwbatch):
        context = xbatch[b:t+1]
        target = ybatch[b, t]
        #print(f"input: {context.tolist()} target: {target}")

______


In [180]:
class LayerNorm1d: # (copy pasted, normalizes rows instead of columns as in BatchNorm)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

#torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [181]:
#create bigram lm
import torch.nn as nn
from torch.nn import functional as F
#torch.manual_seed(1337)
class Head(nn.Module):
    #one head of self attention
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(context_window, context_window)))
        self.dropout = nn.Dropout(dropout)
    '''B, T, C = 4, 8, 32 #batch, time, channels - 4 by 8 arrangement of tokens and each token is currently 32 dimenisional
    #x = torch.randn(B, T, C)

    #single attention head
    head_size = 16
    query = nn.Linear(C, head_size, bias = False)
    key = nn.Linear(C, head_size, bias = False)
    value = nn.Linear(C, head_size, bias = False)

    #tril = torch.tril(torch.ones(T, T))
    #wei = torch.zeros((T, T))

    the following line is very important. in autoregressive settings like LLMS, you mask future tokens from being used in the
    context of a previous token. Hence why the masked fill line is present. In cases like sentiment
    analysis, you may not want information to be masked, and rather have every token talk to every other
    token in the sequence, so you would just delete the directly following masked fill line. 
    The autoregressive method with masked fill is known as a decoder attention block, sentiment like 
    full context without traingular masking is known as an encoder attention block
    wei = wei.masked_fill(tril == 0, float('-inf'))


    wei - F.softmax(wei, dim = -1)
    v = value(x)
    out = wei @ v
    #out = wei @ x
    out.shape'''
    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x) #(B, T, 16) where 16 is head size aka B, T, C
        q = self.query(x) #(B, T, 16)
        wei = q @ k.transpose(-2, -1) * C**-0.5 #only transpose the last two since we need the B vector, normalize it
        #--> (B, T, 16) @ (B, 16, T) --> (B, T, T) (makes the weights data dependent and not just uniform)
        '''the following line is very important. in autoregressive settings like LLMS, you mask future tokens from being used in the
        context of a previous token. Hence why the masked fill line is present. In cases like sentiment
        analysis, you may not want information to be masked, and rather have every token talk to every other
        token in the sequence, so you would just delete the directly following masked fill line. 
        The autoregressive method with masked fill is known as a decoder attention block, sentiment like 
        full context without traingular masking is known as an encoder attention block'''
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1) #(B, T, T)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out  
class MultiHeadAttention(nn.Module):
    #multiple self-attention heads running in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) #linear transformation of the outcome of the concatennation within MHA
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1) #concatenate results of the parallelly running attention heads over the channel dimension
        out = self.dropout(self.proj(out))
        return out
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), #4 multiplied pursuant to the original Attnetion is all you need paper 512 --> 2048 --> 512
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), #residual layer
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
class Block(nn.Module):
    #communication then computation
    def __init__(self, n_embd, n_head):
        #n_head = number of heads we want
        super().__init__()
        head_size = n_embd //n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) #create a vocab by vocab sized embedding table - each token reads off the logits for the following token based on this table
        #box 1: token encoding is complete, now box2: positional encoddings
        self.position_embedding_table = nn.Embedding(context_window, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd)
        )
        #self.selfattentionhead = MultiHeadAttention(4, n_embd//4) #4 heads of 8-dimensional self-attention
        #self.ffwd = FeedForward(n_embd)
        self.languagemodellinghead = nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor of integers
        B, T = idx.shape
        
        token_embeddings = self.token_embedding_table(idx) #(Batch, Time, Channel) AKA B,T,C where B = batchsize (4), T = cwbatch = context window = 8, channel = vocab_size (65)
        positional_embeddings = self.position_embedding_table(torch.arange(T))
        x = token_embeddings + positional_embeddings  #(B, T, C)
        #x = self.selfattentionhead(x) #apply one head of self-attention outputting answer in format (B,T,C)
        #x = self.ffwd(x) #(B, T, C)
        x = self.blocks(x) #(B, T, C)
        logits = self.languagemodellinghead(x) #(B, T, vocabsize) - this step applies the decoder language model we have to create logits from the output of the previous line
        if targets is None: #optional to syncronize with the generate call through self (self in genertate calls forward)
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #merge B and T dimensions into one dimensional vector in order to match pytorch requirement for C to be the second parameter for the cross entropy fn
            loss = F.cross_entropy(logits, targets) #assess accuracy of logits (predictions) relative to the targets - correct dimension of logits should be a high number, rest very low number (essentaily unnormalized prob dists.)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        #idx is (B,T) array of indicies here - represents the current context of some characters in some batch
        #generates one by one increase in tokens and appends this to the running sequence - go from B x T to B x T + 1, B x T + 2
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -batchsize:] #crop idx to the last batch_size number of tokens to ensure positional encodings are within rangae
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :] #only last time step to become (B, C)
            probs = F.softmax(logits, dim = -1) #softmax --> prb dist. (B, C)
            idx_next = torch.multinomial(probs, num_samples = 1) #(B, 1 - single prediction for what comes next in each of the batch dimensions)
            idx = torch.cat((idx, idx_next), dim = 1) #(B, T+1) #extra for bigram model since that relies on prev token prediction and this concatenation 
            #passes in the entire accumulated string into the model each time to predict the next token, but this generate function is meant to be generalizable 
            #to different models that want/require more context than just the preceding character to predict the next character (can swap character/token here depending on tokenization scheme used)

        return idx
m = BigramLM(vocab_size)
logits, loss = m(xbatch, ybatch)
print(logits.shape)
print(loss)
print("expected loss for negative log likelihood of -ln(1/65) ~ 4.17")
#generates logits (scores) for every one of the 4 by 8 positions

# idx portion: batch and time are 1 --> tensor is 1 x 1 and holdds a 0. 0 kicks off the generation (represents new line char)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 100)[0].tolist())) #indexing into 0th row indexes into the single batch dimension that exists

torch.Size([2048, 65])
tensor(4.2550, grad_fn=<NllLossBackward0>)
expected loss for negative log likelihood of -ln(1/65) ~ 4.17

3WfPa
Z..wO aVc
i$g;tAhCA;t?!:TtGgyt-zt$3iZUDWoAUrMW!nuPW.oMxzujsqHfxqk'hU,EjoMTzHOgW.ikju;3SxlTXLCf


In [182]:
#create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [183]:
#batch_size = 32 update to be bigger
for steps in range(5000):
    xbatch, ybatch = get_batch('train')
    logits, loss = m(xbatch, ybatch)
    optimizer.zero_grad(set_to_none = True) #zero out gradients from the previous step 
    loss.backward() #get those gradients for all the parameters
    optimizer.step() #using the gradients to update the parameters

print(loss.item())

1.5416239500045776


In [184]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype = torch.long), max_new_tokens = 2000)[0].tolist())) #indexing into 0th row indexes into the single batch dimension that exists


Our marrs
For good comt child pritse the worst:
You would balievaltes.
Is this envictor finst like a puned offence:
He the guits welge wells herefore.
Firstamed it ado much tears fortended,
Ichar deukin fouguits dostingfit all cousing.
Theself.

ANGELIZABET:
Thou stand in the shonour, leavene you he have child aduction
Which intisforess fully fraith hate charmful LadyY

Now:
Extand spearous.

DUCHESS OF YORK:
Why say, there mee protriansous-upoes,
I kine.

GREEN:
Sir, Herein you hencle, is defio,
Sun peach; and and mond our each'd
Of did vithal, as I tell granion;
The prince; and they good gifts,
Oxforted with wordst burn be gody,
To comfroance thee be Richal spuersate,
That good like by forken this should,
My layer's sbeen stand dons, but itranchands dighance.

NORTENSIO:
For
That on inf oh hears and her maymany you?

WARWICK:
My lord, thou wantstage
Asse king, this all this that beenf;
It with Mercius, whehink, he againly day
So hhis spoking, my freme, these shame, on thy patar?
Goo

In [185]:
#masked self attention below

In [186]:
 #v1
B, T, C = 4, 8 , 2
x = torch.randn(B, T, C)
xbagofwords = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # shape (t,C)
        xbagofwords[b, t] = torch.mean(xprev, 0)
#simple averaging C over all prev words - not effective since very lossy, and inefficient without matrix math
x[0]

tensor([[ 1.2934, -1.2048],
        [ 1.3275, -0.5738],
        [-0.8724,  0.9453],
        [ 1.0937, -0.9218],
        [-2.5540, -0.4727],
        [ 1.6102, -1.1374],
        [-1.8820, -0.7011],
        [ 0.9065,  0.8109]])

In [187]:
xbagofwords[0]
#average of all previous moving down

tensor([[ 1.2934, -1.2048],
        [ 1.3105, -0.8893],
        [ 0.5828, -0.2778],
        [ 0.7105, -0.4388],
        [ 0.0576, -0.4456],
        [ 0.3164, -0.5609],
        [ 0.0023, -0.5809],
        [ 0.1154, -0.4069]])

In [188]:
#v2
weights = torch.tril(torch.ones(T, T))
#create triangular matrix for averaging in conjunction with next step
weights = weights/weights.sum(1, keepdim = True)  #rows sum to 1
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [189]:
xbagofwords2 = weights @ x #(T,T) @ (B, T, C) --> pytorch will add a batch dimension (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(xbagofwords, xbagofwords2) #now will be the same after the prev step

True

In [190]:
 #v3 using softmax
tril = torch.tril(torch.ones(T, T)) #lower triangular matrix of 1s
weights = torch.zeros((T, T)) #all 0s initially
weights = weights.masked_fill(tril == 0, float('-inf')) #assign tril 0s to be - infinity
weights = F.softmax(weights, dim = -1) #take softmax along every single row - normalizes and spits back the same matrix for xbagofwords and xbagofwords3
xbagofwords3 = weights @ x
torch.allclose(xbagofwords, xbagofwords3)
#this is the preferred version for self-attention since the pd generated by the softmax demonstartes impact of certain tokens communicating with others to influence their context
#the weights are data-dependent, goes to the Q, K, V, context-rich vector mechanisms in the multihead triangularly masked self attention mechanism



True

In [191]:
#v4: self attention
'''torch.manual_seed(1337)
class Head(nn.Module):
    #one head of self attention
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(batch_size, batch_size)))'''
'''B, T, C = 4, 8, 32 #batch, time, channels - 4 by 8 arrangement of tokens and each token is currently 32 dimenisional
#x = torch.randn(B, T, C)

#single attention head
head_size = 16
query = nn.Linear(C, head_size, bias = False)
key = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)

#tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T, T))

the following line is very important. in autoregressive settings like LLMS, you mask future tokens from being used in the
context of a previous token. Hence why the masked fill line is present. In cases like sentiment
analysis, you may not want information to be masked, and rather have every token talk to every other
token in the sequence, so you would just delete the directly following masked fill line. 
The autoregressive method with masked fill is known as a decoder attention block, sentiment like 
full context without traingular masking is known as an encoder attention block
wei = wei.masked_fill(tril == 0, float('-inf'))


wei - F.softmax(wei, dim = -1)
v = value(x)
out = wei @ v
#out = wei @ x
out.shape'''
'''def forward(self,x):
    B,T,C = x.shape
    k = self.key(x) #(B, T, 16) where 16 is head size aka B, T, C
    q = self.query(x) #(B, T, 16)
    wei = q @ k.transpose(-2, -1) * C**-0.5 #only transpose the last two since we need the B vector, normalize it
#--> (B, T, 16) @ (B, 16, T) --> (B, T, T) (makes the weights data dependent and not just uniform)
    the following line is very important. in autoregressive settings like LLMS, you mask future tokens from being used in the
    context of a previous token. Hence why the masked fill line is present. In cases like sentiment
    analysis, you may not want information to be masked, and rather have every token talk to every other
    token in the sequence, so you would just delete the directly following masked fill line. 
    The autoregressive method with masked fill is known as a decoder attention block, sentiment like 
    full context without traingular masking is known as an encoder attention block
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim = -1) #(B, T, T)
    v = self.value(x)
    out = wei @ v
    return out  '''
    

"def forward(self,x):\n    B,T,C = x.shape\n    k = self.key(x) #(B, T, 16) where 16 is head size aka B, T, C\n    q = self.query(x) #(B, T, 16)\n    wei = q @ k.transpose(-2, -1) * C**-0.5 #only transpose the last two since we need the B vector, normalize it\n#--> (B, T, 16) @ (B, 16, T) --> (B, T, T) (makes the weights data dependent and not just uniform)\n    the following line is very important. in autoregressive settings like LLMS, you mask future tokens from being used in the\n    context of a previous token. Hence why the masked fill line is present. In cases like sentiment\n    analysis, you may not want information to be masked, and rather have every token talk to every other\n    token in the sequence, so you would just delete the directly following masked fill line. \n    The autoregressive method with masked fill is known as a decoder attention block, sentiment like \n    full context without traingular masking is known as an encoder attention block\n    wei = wei.masked_fi

In [192]:
model = BigramLM(vocab_size)
print(sum(p.numel() for p in model.parameters()))

618817
